# Intro

This project takes 2 location history files and outputs 2 things:

1. For each day how many time the people spent together in seconds
2. The observations with timestamp and location for which there is a *match* between two people.
    * A match is defined as having a corresponding timestamp between two people, with two locations which are not too far from each other.

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import numpy as np

In [3]:
import pandas as pd
import json
import geopy.distance
import matplotlib.pyplot as plt

In [4]:
pd.set_option("display.precision", 9)

# Create example to run everything on

In [5]:
with open('Klaas-filtered.json') as json_file:
    df1 = json.load(json_file)
df1 = pd.DataFrame(df1)
df1 = df1.drop("activity", 1)
df1 = df1[df1['timestampMs'] < '1572650160000']
df1

,timestampMs,latitudeE7,longitudeE7,accuracy,altitude,verticalAccuracy,velocity,heading
0,1564621610402,508619179,43359986,14,79.0,3.0,NaN,NaN
1,1564623771349,508619471,43359965,15,75.0,3.0,NaN,NaN
2,1564624761191,508619535,43359980,16,79.0,3.0,NaN,NaN
3,1564625362345,508619469,43360118,14,79.0,3.0,NaN,NaN
4,1564627523336,508619358,43359998,15,79.0,3.0,NaN,NaN
...,...,...,...,...,...,...,...,...
38307,1572649082673,46005019,-740691845,50,NaN,NaN,NaN,NaN
38308,1572649202734,46005019,-740691845,50,NaN,NaN,NaN,NaN
38309,1572649322751,46005019,-740691845,50,NaN,NaN,NaN,NaN
38310,1572649478731,46005019,-740691845,50,NaN,NaN,NaN,NaN


In [6]:
with open('Thomas-filtered.json') as json_file:
    df2 = json.load(json_file)
df2 = pd.DataFrame(df2)
df2

,timestampMs,latitudeE7,longitudeE7,accuracy,altitude,verticalAccuracy,velocity,heading
0,1564637041886,510350641,37339686,65,2.0,10.0,NaN,NaN
1,1564640697015,510341940,37362204,24,2.0,3.0,1.0,NaN
2,1564640875904,510322913,37339917,16,0.0,3.0,1.0,NaN
3,1564641018721,510325417,37342550,65,3.0,10.0,NaN,NaN
4,1564641267355,510300225,37378699,65,4.0,10.0,NaN,NaN
...,...,...,...,...,...,...,...,...
3362,1571585366999,489340639,23920436,32,20.0,12.0,11.0,89.0
3363,1571585446999,489346679,23989600,32,-5.0,16.0,7.0,76.0
3364,1571585526998,489370248,24083527,32,24.0,15.0,9.0,71.0
3365,1571585606998,489390858,24167024,32,38.0,36.0,8.0,68.0


**Convert ts col to readable format**

In [11]:
df1['timestampMs'] = pd.to_datetime(df1['timestampMs'], unit = 'ms')
df1.rename(columns = {'timestampMs':'timestamp'}, inplace = True)

df2['timestampMs'] = pd.to_datetime(df2['timestampMs'], unit = 'ms')
df2.rename(columns = {'timestampMs':'timestamp'}, inplace = True)

In [13]:
# lat en long
df1["latitudeE7"] = df1["latitudeE7"].div(10**7).round(9)
df1["longitudeE7"] = df1["longitudeE7"].div(10**7).round(9)

df1.rename(columns = {'latitudeE7':'latitude',
                       'longitudeE7':'longitude'}, inplace = True)

df2["latitudeE7"] = df2["latitudeE7"].div(10**7).round(9)
df2["longitudeE7"] = df2["longitudeE7"].div(10**7).round(9)

df2.rename(columns = {'latitudeE7':'latitude',
                       'longitudeE7':'longitude'}, inplace = True)

In [15]:
# drop other columns
df1 = df1.drop(columns = ['accuracy', 'altitude', 'verticalAccuracy', 'velocity', 'heading'])
df2 = df2.drop(columns = ['accuracy', 'altitude', 'verticalAccuracy', 'velocity', 'heading'])

## Run the code

In [18]:
df1
df2

,timestamp,latitude,longitude
0,2019-08-01 01:06:50.402,50.8619179,4.3359986
1,2019-08-01 01:42:51.349,50.8619471,4.3359965
2,2019-08-01 01:59:21.191,50.8619535,4.3359980
3,2019-08-01 02:09:22.345,50.8619469,4.3360118
4,2019-08-01 02:45:23.336,50.8619358,4.3359998
...,...,...,...
38307,2019-11-01 22:58:02.673,4.6005019,-74.0691845
38308,2019-11-01 23:00:02.734,4.6005019,-74.0691845
38309,2019-11-01 23:02:02.751,4.6005019,-74.0691845
38310,2019-11-01 23:04:38.731,4.6005019,-74.0691845


,timestamp,latitude,longitude
0,2019-08-01 05:24:01.886,51.0350641,3.7339686
1,2019-08-01 06:24:57.015,51.0341940,3.7362204
2,2019-08-01 06:27:55.904,51.0322913,3.7339917
3,2019-08-01 06:30:18.721,51.0325417,3.7342550
4,2019-08-01 06:34:27.355,51.0300225,3.7378699
...,...,...,...
3362,2019-10-20 15:29:26.999,48.9340639,2.3920436
3363,2019-10-20 15:30:46.999,48.9346679,2.3989600
3364,2019-10-20 15:32:06.998,48.9370248,2.4083527
3365,2019-10-20 15:33:26.998,48.9390858,2.4167024


In [19]:
# do full merge of two df's
# every ts of df1 should be as many times duplicated for every row of df2

# merging on new key to get full cartesian product
# https://stackoverflow.com/questions/13269890/cartesian-product-in-pandas
# create new key column in each df

df1['key'] = 1
df2['key'] = 1

result = pd.merge(df1, df2, on = 'key')
# result = pd.merge(df1[['key', 'timestamp']], df2[['key', 'timestamp']], on = 'key')

result = result.drop(columns='key')

In [20]:
result

,timestamp_x,latitude_x,longitude_x,timestamp_y,latitude_y,longitude_y
0,2019-08-01 01:06:50.402,50.8619179,4.3359986,2019-08-01 05:24:01.886,51.0350641,3.7339686
1,2019-08-01 01:06:50.402,50.8619179,4.3359986,2019-08-01 06:24:57.015,51.0341940,3.7362204
2,2019-08-01 01:06:50.402,50.8619179,4.3359986,2019-08-01 06:27:55.904,51.0322913,3.7339917
3,2019-08-01 01:06:50.402,50.8619179,4.3359986,2019-08-01 06:30:18.721,51.0325417,3.7342550
4,2019-08-01 01:06:50.402,50.8619179,4.3359986,2019-08-01 06:34:27.355,51.0300225,3.7378699
...,...,...,...,...,...,...
128996499,2019-11-01 23:06:45.066,4.6005019,-74.0691845,2019-10-20 15:29:26.999,48.9340639,2.3920436
128996500,2019-11-01 23:06:45.066,4.6005019,-74.0691845,2019-10-20 15:30:46.999,48.9346679,2.3989600
128996501,2019-11-01 23:06:45.066,4.6005019,-74.0691845,2019-10-20 15:32:06.998,48.9370248,2.4083527
128996502,2019-11-01 23:06:45.066,4.6005019,-74.0691845,2019-10-20 15:33:26.998,48.9390858,2.4167024


In [21]:
max_time = 10 
max_time = pd.to_timedelta(max_time, unit = 'minutes')

In [22]:
# filter combined df on upper and lower bound
result = result.loc[(result['timestamp_x'] <= result['timestamp_y'] + max_time) & (result['timestamp_x'] >= result['timestamp_y'] - max_time)]

In [1]:
if result.empty == True:
    raise ValueError('There are no overlapping matches based on GPS traces. \nTHIS IS SAD')

NameError: name 'result' is not defined

In [25]:
result

,timestamp_x,latitude_x,longitude_x,timestamp_y,latitude_y,longitude_y
60606,2019-08-01 05:20:28.518,50.8619474,4.3359985,2019-08-01 05:24:01.886,51.0350641,3.7339686
63973,2019-08-01 05:26:31.305,50.8619292,4.3359912,2019-08-01 05:24:01.886,51.0350641,3.7339686
67340,2019-08-01 05:32:32.325,50.8619361,4.3359995,2019-08-01 05:24:01.886,51.0350641,3.7339686
111112,2019-08-01 06:15:50.628,50.8618882,4.3359122,2019-08-01 06:24:57.015,51.0341940,3.7362204
114479,2019-08-01 06:20:50.721,50.8618793,4.3359543,2019-08-01 06:24:57.015,51.0341940,3.7362204
...,...,...,...,...,...,...
106060204,2019-10-16 16:28:38.010,51.0173066,4.4776110,2019-10-16 16:21:48.999,51.0351943,3.7357462
106060205,2019-10-16 16:28:38.010,51.0173066,4.4776110,2019-10-16 16:23:57.803,51.0351859,3.7342722
106063571,2019-10-16 16:30:39.111,51.0173070,4.4776119,2019-10-16 16:21:48.999,51.0351943,3.7357462
106063572,2019-10-16 16:30:39.111,51.0173070,4.4776119,2019-10-16 16:23:57.803,51.0351859,3.7342722


In [26]:
# count how many matches in interval between df1 and df2 (groupbyby ts of df1)
# for each timestamp of df1, how many rows are there
new_colname = 'count_overlaps_in_int'

result[new_colname] = result.groupby('timestamp_x')['timestamp_x'].transform('count')

<ipython-input-26-a135545f0540>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result[new_colname] = result.groupby('timestamp_x')['timestamp_x'].transform('count')


In [27]:
result

,timestamp_x,latitude_x,longitude_x,timestamp_y,latitude_y,longitude_y,count_overlaps_in_int
60606,2019-08-01 05:20:28.518,50.8619474,4.3359985,2019-08-01 05:24:01.886,51.0350641,3.7339686,1
63973,2019-08-01 05:26:31.305,50.8619292,4.3359912,2019-08-01 05:24:01.886,51.0350641,3.7339686,1
67340,2019-08-01 05:32:32.325,50.8619361,4.3359995,2019-08-01 05:24:01.886,51.0350641,3.7339686,1
111112,2019-08-01 06:15:50.628,50.8618882,4.3359122,2019-08-01 06:24:57.015,51.0341940,3.7362204,1
114479,2019-08-01 06:20:50.721,50.8618793,4.3359543,2019-08-01 06:24:57.015,51.0341940,3.7362204,3
...,...,...,...,...,...,...,...
106060204,2019-10-16 16:28:38.010,51.0173066,4.4776110,2019-10-16 16:21:48.999,51.0351943,3.7357462,2
106060205,2019-10-16 16:28:38.010,51.0173066,4.4776110,2019-10-16 16:23:57.803,51.0351859,3.7342722,2
106063571,2019-10-16 16:30:39.111,51.0173070,4.4776119,2019-10-16 16:21:48.999,51.0351943,3.7357462,2
106063572,2019-10-16 16:30:39.111,51.0173070,4.4776119,2019-10-16 16:23:57.803,51.0351859,3.7342722,2


In [28]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25671 entries, 60606 to 106066939
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   timestamp_x            25671 non-null  datetime64[ns]
 1   latitude_x             25671 non-null  float64       
 2   longitude_x            25671 non-null  float64       
 3   timestamp_y            25671 non-null  datetime64[ns]
 4   latitude_y             25671 non-null  float64       
 5   longitude_y            25671 non-null  float64       
 6   count_overlaps_in_int  25671 non-null  int64         
dtypes: datetime64[ns](2), float64(4), int64(1)
memory usage: 1.6 MB


In [29]:
# calc dist and filter on that
for lab, row in result.iterrows():
    result.loc[lab, 'dist'] = geopy.distance.distance((row['latitude_x'], row['longitude_x']),(row['latitude_y'], row['longitude_y'])).m

c:\users\u0100050\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
c:\users\u0100050\appdata\local\programs\python\python38\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [30]:
result

,timestamp_x,latitude_x,longitude_x,timestamp_y,latitude_y,longitude_y,count_overlaps_in_int,dist
60606,2019-08-01 05:20:28.518,50.8619474,4.3359985,2019-08-01 05:24:01.886,51.0350641,3.7339686,1,46484.811531111
63973,2019-08-01 05:26:31.305,50.8619292,4.3359912,2019-08-01 05:24:01.886,51.0350641,3.7339686,1,46485.190997363
67340,2019-08-01 05:32:32.325,50.8619361,4.3359995,2019-08-01 05:24:01.886,51.0350641,3.7339686,1,46485.400970788
111112,2019-08-01 06:15:50.628,50.8618882,4.3359122,2019-08-01 06:24:57.015,51.0341940,3.7362204,1,46298.277681280
114479,2019-08-01 06:20:50.721,50.8618793,4.3359543,2019-08-01 06:24:57.015,51.0341940,3.7362204,3,46301.384361709
...,...,...,...,...,...,...,...,...
106060204,2019-10-16 16:28:38.010,51.0173066,4.4776110,2019-10-16 16:21:48.999,51.0351943,3.7357462,2,52085.602649285
106060205,2019-10-16 16:28:38.010,51.0173066,4.4776110,2019-10-16 16:23:57.803,51.0351859,3.7342722,2,52188.908047730
106063571,2019-10-16 16:30:39.111,51.0173070,4.4776119,2019-10-16 16:21:48.999,51.0351943,3.7357462,2,52085.663820756
106063572,2019-10-16 16:30:39.111,51.0173070,4.4776119,2019-10-16 16:23:57.803,51.0351859,3.7342722,2,52188.969223145


In [31]:
# filter on maximum distance
max_dist = 20

result = result.loc[result['dist'] < max_dist]

result

,timestamp_x,latitude_x,longitude_x,timestamp_y,latitude_y,longitude_y,count_overlaps_in_int,dist
52442900,2019-09-09 06:39:14.097,52.2525362,0.7138794,2019-09-09 06:46:51.199,52.2525704,0.7138478,2,4.374792876
52442901,2019-09-09 06:39:14.097,52.2525362,0.7138794,2019-09-09 06:48:59.484,52.2526024,0.7138553,2,7.547835973
52446267,2019-09-09 06:40:55.850,52.2525082,0.7138945,2019-09-09 06:46:51.199,52.2525704,0.7138478,2,7.620557842
52446268,2019-09-09 06:40:55.850,52.2525082,0.7138945,2019-09-09 06:48:59.484,52.2526024,0.7138553,2,10.818282131
52449634,2019-09-09 06:45:08.096,52.2525082,0.7138945,2019-09-09 06:46:51.199,52.2525704,0.7138478,2,7.620557842
...,...,...,...,...,...,...,...,...
58638480,2019-09-15 08:46:13.779,51.4897612,-0.1157898,2019-09-15 08:51:56.791,51.4898393,-0.1155317,1,19.921482811
58827036,2019-09-15 11:04:09.436,51.5309645,-0.1257586,2019-09-15 11:03:42.148,51.5309472,-0.1255995,4,11.207023835
58833770,2019-09-15 11:08:13.895,51.5308573,-0.1258008,2019-09-15 11:03:42.148,51.5309472,-0.1255995,4,17.180610921
58840504,2019-09-15 11:11:33.024,51.5309513,-0.1257283,2019-09-15 11:03:42.148,51.5309472,-0.1255995,4,8.949512239


In [32]:
if result.empty == True:
    raise ValueError('There are no overlapping matches based on GPS traces. \nTHIS IS SAD')

In [33]:
# Now filter on time between two observations
result.sort_values("timestamp_x", inplace = True)

# For each two rows, check if the smallest difference between the latest ts of the first 
# row and the earliest ts of the second row is smaller than the max_delta_ts

# We allow this difference to pass over different days for simplicity
# So do not calculate this by day yet
result['ts_latest'] = result.loc[:, ['timestamp_x', 'timestamp_y']].max(axis=1)
result['ts_earliest'] = result.loc[:, ['timestamp_x', 'timestamp_y']].min(axis=1)

# Calculate the difference between each two rows: the ts_latest of the 1st row, the 
# earliest of the 2nd row

# move ts_earliest up one row
result['ts_earliest_moved_up'] = result['ts_earliest'].shift(periods = -1)

# calculate diff
# result['ts_smallest_delta'] = result['ts_latest'] - result['ts_earliest_moved_up']
# Weird issue with day difference?
# Just take the difference of the latest value 
result['ts_smallest_delta'] = result.loc[:, ['ts_latest', 'ts_earliest_moved_up']].max(axis=1) - result.loc[:, ['ts_latest', 'ts_earliest_moved_up']].min(axis=1)

result
# result['ts_earliest_moved_up'][5]
# print(str(result['ts_earliest_moved_up'][5]))



<ipython-input-33-f52b5caedb65>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result.sort_values("timestamp_x", inplace = True)
<ipython-input-33-f52b5caedb65>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['ts_latest'] = result.loc[:, ['timestamp_x', 'timestamp_y']].max(axis=1)
<ipython-input-33-f52b5caedb65>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guid

,timestamp_x,latitude_x,longitude_x,timestamp_y,latitude_y,longitude_y,count_overlaps_in_int,dist,ts_latest,ts_earliest,ts_earliest_moved_up,ts_smallest_delta
52442900,2019-09-09 06:39:14.097,52.2525362,0.7138794,2019-09-09 06:46:51.199,52.2525704,0.7138478,2,4.374792876,2019-09-09 06:46:51.199,2019-09-09 06:39:14.097,2019-09-09 06:39:14.097,00:07:37.102000
52442901,2019-09-09 06:39:14.097,52.2525362,0.7138794,2019-09-09 06:48:59.484,52.2526024,0.7138553,2,7.547835973,2019-09-09 06:48:59.484,2019-09-09 06:39:14.097,2019-09-09 06:40:55.850,00:08:03.634000
52446267,2019-09-09 06:40:55.850,52.2525082,0.7138945,2019-09-09 06:46:51.199,52.2525704,0.7138478,2,7.620557842,2019-09-09 06:46:51.199,2019-09-09 06:40:55.850,2019-09-09 06:40:55.850,00:05:55.349000
52446268,2019-09-09 06:40:55.850,52.2525082,0.7138945,2019-09-09 06:48:59.484,52.2526024,0.7138553,2,10.818282131,2019-09-09 06:48:59.484,2019-09-09 06:40:55.850,2019-09-09 06:45:08.096,00:03:51.388000
52449634,2019-09-09 06:45:08.096,52.2525082,0.7138945,2019-09-09 06:46:51.199,52.2525704,0.7138478,2,7.620557842,2019-09-09 06:46:51.199,2019-09-09 06:45:08.096,2019-09-09 06:45:08.096,00:01:43.103000
...,...,...,...,...,...,...,...,...,...,...,...,...
58638480,2019-09-15 08:46:13.779,51.4897612,-0.1157898,2019-09-15 08:51:56.791,51.4898393,-0.1155317,1,19.921482811,2019-09-15 08:51:56.791,2019-09-15 08:46:13.779,2019-09-15 11:03:42.148,02:11:45.357000
58827036,2019-09-15 11:04:09.436,51.5309645,-0.1257586,2019-09-15 11:03:42.148,51.5309472,-0.1255995,4,11.207023835,2019-09-15 11:04:09.436,2019-09-15 11:03:42.148,2019-09-15 11:03:42.148,00:00:27.288000
58833770,2019-09-15 11:08:13.895,51.5308573,-0.1258008,2019-09-15 11:03:42.148,51.5309472,-0.1255995,4,17.180610921,2019-09-15 11:08:13.895,2019-09-15 11:03:42.148,2019-09-15 11:03:42.148,00:04:31.747000
58840504,2019-09-15 11:11:33.024,51.5309513,-0.1257283,2019-09-15 11:03:42.148,51.5309472,-0.1255995,4,8.949512239,2019-09-15 11:11:33.024,2019-09-15 11:03:42.148,2019-09-15 11:03:42.148,00:07:50.876000


In [34]:
result.shape

(152, 12)

In [35]:
max_delta_ts = 60
max_delta_ts = pd.to_timedelta(max_delta_ts, unit = 'minutes')

# filter on max_delta_ts
# Note that the last row also gets deleted (NaT)
result = result.loc[result['ts_smallest_delta'] < max_delta_ts]


In [36]:
result.shape

(135, 12)

In [50]:
if result.empty == True:
    raise ValueError('There are no overlapping matches based on GPS traces. \nTHIS IS SAD')

In [38]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135 entries, 52442900 to 58843871
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype          
---  ------                 --------------  -----          
 0   timestamp_x            135 non-null    datetime64[ns] 
 1   latitude_x             135 non-null    float64        
 2   longitude_x            135 non-null    float64        
 3   timestamp_y            135 non-null    datetime64[ns] 
 4   latitude_y             135 non-null    float64        
 5   longitude_y            135 non-null    float64        
 6   count_overlaps_in_int  135 non-null    int64          
 7   dist                   135 non-null    float64        
 8   ts_latest              135 non-null    datetime64[ns] 
 9   ts_earliest            135 non-null    datetime64[ns] 
 10  ts_earliest_moved_up   134 non-null    datetime64[ns] 
 11  ts_smallest_delta      135 non-null    timedelta64[ns]
dtypes: datetime64[ns](5), float64(5), int6

In [39]:
result

,timestamp_x,latitude_x,longitude_x,timestamp_y,latitude_y,longitude_y,count_overlaps_in_int,dist,ts_latest,ts_earliest,ts_earliest_moved_up,ts_smallest_delta
52442900,2019-09-09 06:39:14.097,52.2525362,0.7138794,2019-09-09 06:46:51.199,52.2525704,0.7138478,2,4.374792876,2019-09-09 06:46:51.199,2019-09-09 06:39:14.097,2019-09-09 06:39:14.097,00:07:37.102000
52442901,2019-09-09 06:39:14.097,52.2525362,0.7138794,2019-09-09 06:48:59.484,52.2526024,0.7138553,2,7.547835973,2019-09-09 06:48:59.484,2019-09-09 06:39:14.097,2019-09-09 06:40:55.850,00:08:03.634000
52446267,2019-09-09 06:40:55.850,52.2525082,0.7138945,2019-09-09 06:46:51.199,52.2525704,0.7138478,2,7.620557842,2019-09-09 06:46:51.199,2019-09-09 06:40:55.850,2019-09-09 06:40:55.850,00:05:55.349000
52446268,2019-09-09 06:40:55.850,52.2525082,0.7138945,2019-09-09 06:48:59.484,52.2526024,0.7138553,2,10.818282131,2019-09-09 06:48:59.484,2019-09-09 06:40:55.850,2019-09-09 06:45:08.096,00:03:51.388000
52449634,2019-09-09 06:45:08.096,52.2525082,0.7138945,2019-09-09 06:46:51.199,52.2525704,0.7138478,2,7.620557842,2019-09-09 06:46:51.199,2019-09-09 06:45:08.096,2019-09-09 06:45:08.096,00:01:43.103000
...,...,...,...,...,...,...,...,...,...,...,...,...
58635113,2019-09-15 08:43:11.727,51.4897612,-0.1157898,2019-09-15 08:51:56.791,51.4898393,-0.1155317,1,19.921482811,2019-09-15 08:51:56.791,2019-09-15 08:43:11.727,2019-09-15 08:46:13.779,00:05:43.012000
58827036,2019-09-15 11:04:09.436,51.5309645,-0.1257586,2019-09-15 11:03:42.148,51.5309472,-0.1255995,4,11.207023835,2019-09-15 11:04:09.436,2019-09-15 11:03:42.148,2019-09-15 11:03:42.148,00:00:27.288000
58833770,2019-09-15 11:08:13.895,51.5308573,-0.1258008,2019-09-15 11:03:42.148,51.5309472,-0.1255995,4,17.180610921,2019-09-15 11:08:13.895,2019-09-15 11:03:42.148,2019-09-15 11:03:42.148,00:04:31.747000
58840504,2019-09-15 11:11:33.024,51.5309513,-0.1257283,2019-09-15 11:03:42.148,51.5309472,-0.1255995,4,8.949512239,2019-09-15 11:11:33.024,2019-09-15 11:03:42.148,2019-09-15 11:03:42.148,00:07:50.876000


In [40]:
# For two 'hits' of observations who are in the allowed max_delta_ts
# For each day
# Calculate the time difference between each two subsequent hits
# For the first row take the earliest ts
# For the second row take the latest ts
# Calculate the difference between those
# And add to total_time_together by day

# Calculate the difference between each two rows: the ts_latest of the 1st row, the 
# earliest of the 2nd row

# move ts_earliest up one row
result['ts_latest_moved_up'] = result['ts_latest'].shift(periods = -1)

# calculate diff
result['ts_delta_since_previous_match'] = result.loc[:, ['ts_latest_moved_up', 'ts_earliest']].max(axis=1) - result.loc[:, ['ts_latest_moved_up', 'ts_earliest']].min(axis=1)

result


<ipython-input-40-c601c6e36882>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['ts_latest_moved_up'] = result['ts_latest'].shift(periods = -1)
<ipython-input-40-c601c6e36882>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['ts_delta_since_previous_match'] = result.loc[:, ['ts_latest_moved_up', 'ts_earliest']].max(axis=1) - result.loc[:, ['ts_latest_moved_up', 'ts_earliest']].min(axis=1)


,timestamp_x,latitude_x,longitude_x,timestamp_y,latitude_y,longitude_y,count_overlaps_in_int,dist,ts_latest,ts_earliest,ts_earliest_moved_up,ts_smallest_delta,ts_latest_moved_up,ts_delta_since_previous_match
52442900,2019-09-09 06:39:14.097,52.2525362,0.7138794,2019-09-09 06:46:51.199,52.2525704,0.7138478,2,4.374792876,2019-09-09 06:46:51.199,2019-09-09 06:39:14.097,2019-09-09 06:39:14.097,00:07:37.102000,2019-09-09 06:48:59.484,00:09:45.387000
52442901,2019-09-09 06:39:14.097,52.2525362,0.7138794,2019-09-09 06:48:59.484,52.2526024,0.7138553,2,7.547835973,2019-09-09 06:48:59.484,2019-09-09 06:39:14.097,2019-09-09 06:40:55.850,00:08:03.634000,2019-09-09 06:46:51.199,00:07:37.102000
52446267,2019-09-09 06:40:55.850,52.2525082,0.7138945,2019-09-09 06:46:51.199,52.2525704,0.7138478,2,7.620557842,2019-09-09 06:46:51.199,2019-09-09 06:40:55.850,2019-09-09 06:40:55.850,00:05:55.349000,2019-09-09 06:48:59.484,00:08:03.634000
52446268,2019-09-09 06:40:55.850,52.2525082,0.7138945,2019-09-09 06:48:59.484,52.2526024,0.7138553,2,10.818282131,2019-09-09 06:48:59.484,2019-09-09 06:40:55.850,2019-09-09 06:45:08.096,00:03:51.388000,2019-09-09 06:46:51.199,00:05:55.349000
52449634,2019-09-09 06:45:08.096,52.2525082,0.7138945,2019-09-09 06:46:51.199,52.2525704,0.7138478,2,7.620557842,2019-09-09 06:46:51.199,2019-09-09 06:45:08.096,2019-09-09 06:45:08.096,00:01:43.103000,2019-09-09 06:48:59.484,00:03:51.388000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58635113,2019-09-15 08:43:11.727,51.4897612,-0.1157898,2019-09-15 08:51:56.791,51.4898393,-0.1155317,1,19.921482811,2019-09-15 08:51:56.791,2019-09-15 08:43:11.727,2019-09-15 08:46:13.779,00:05:43.012000,2019-09-15 11:04:09.436,02:20:57.709000
58827036,2019-09-15 11:04:09.436,51.5309645,-0.1257586,2019-09-15 11:03:42.148,51.5309472,-0.1255995,4,11.207023835,2019-09-15 11:04:09.436,2019-09-15 11:03:42.148,2019-09-15 11:03:42.148,00:00:27.288000,2019-09-15 11:08:13.895,00:04:31.747000
58833770,2019-09-15 11:08:13.895,51.5308573,-0.1258008,2019-09-15 11:03:42.148,51.5309472,-0.1255995,4,17.180610921,2019-09-15 11:08:13.895,2019-09-15 11:03:42.148,2019-09-15 11:03:42.148,00:04:31.747000,2019-09-15 11:11:33.024,00:07:50.876000
58840504,2019-09-15 11:11:33.024,51.5309513,-0.1257283,2019-09-15 11:03:42.148,51.5309472,-0.1255995,4,8.949512239,2019-09-15 11:11:33.024,2019-09-15 11:03:42.148,2019-09-15 11:03:42.148,00:07:50.876000,2019-09-15 11:11:54.643,00:08:12.495000


In [41]:
result.shape

(135, 14)

In [42]:
# Delete last match because there is no 'next' match that we can use to calculate the total time spent together
result.drop(result.tail(1).index,inplace=True) 

c:\users\u0100050\appdata\local\programs\python\python38\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [43]:
result.shape

(134, 14)

In [44]:
# Add column which sums time spent together for each day
result['current_day'] = result['timestamp_x'].dt.date

#result['time_together_today'] = 
output = result.groupby('current_day')['ts_smallest_delta'].sum()

output = pd.DataFrame({'date': output.index, 'time_together': output.values})

output

<ipython-input-44-731412db5e2b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['current_day'] = result['timestamp_x'].dt.date


,date,time_together
0,2019-09-09,03:41:23.498000
1,2019-09-12,00:25:37.599000
2,2019-09-13,05:17:51.378000
3,2019-09-14,04:40:11.522000
4,2019-09-15,00:18:32.923000


In [45]:
result.columns

Index(['timestamp_x', 'latitude_x', 'longitude_x', 'timestamp_y', 'latitude_y',
       'longitude_y', 'count_overlaps_in_int', 'dist', 'ts_latest',
       'ts_earliest', 'ts_earliest_moved_up', 'ts_smallest_delta',
       'ts_latest_moved_up', 'ts_delta_since_previous_match', 'current_day'],
      dtype='object')

**Merge with result to get second type of output and clean up**


In [46]:
# Merge with result to get second type of output and clean up
result = result[['timestamp_x', 'latitude_x', 'longitude_x', 'timestamp_y', 'latitude_y', 'longitude_y', 'count_overlaps_in_int', 'dist', 'current_day']]

result

,timestamp_x,latitude_x,longitude_x,timestamp_y,latitude_y,longitude_y,count_overlaps_in_int,dist,current_day
52442900,2019-09-09 06:39:14.097,52.2525362,0.7138794,2019-09-09 06:46:51.199,52.2525704,0.7138478,2,4.374792876,2019-09-09
52442901,2019-09-09 06:39:14.097,52.2525362,0.7138794,2019-09-09 06:48:59.484,52.2526024,0.7138553,2,7.547835973,2019-09-09
52446267,2019-09-09 06:40:55.850,52.2525082,0.7138945,2019-09-09 06:46:51.199,52.2525704,0.7138478,2,7.620557842,2019-09-09
52446268,2019-09-09 06:40:55.850,52.2525082,0.7138945,2019-09-09 06:48:59.484,52.2526024,0.7138553,2,10.818282131,2019-09-09
52449634,2019-09-09 06:45:08.096,52.2525082,0.7138945,2019-09-09 06:46:51.199,52.2525704,0.7138478,2,7.620557842,2019-09-09
...,...,...,...,...,...,...,...,...,...
57742856,2019-09-14 23:06:03.000,51.4897918,-0.1158281,2019-09-14 22:56:46.626,51.4897181,-0.1156491,1,14.893132608,2019-09-14
58635113,2019-09-15 08:43:11.727,51.4897612,-0.1157898,2019-09-15 08:51:56.791,51.4898393,-0.1155317,1,19.921482811,2019-09-15
58827036,2019-09-15 11:04:09.436,51.5309645,-0.1257586,2019-09-15 11:03:42.148,51.5309472,-0.1255995,4,11.207023835,2019-09-15
58833770,2019-09-15 11:08:13.895,51.5308573,-0.1258008,2019-09-15 11:03:42.148,51.5309472,-0.1255995,4,17.180610921,2019-09-15


In [47]:
output2 = pd.merge(result, output, left_on = 'current_day' , right_on = 'date')
output2 = output2.drop(columns='current_day')

In [48]:
output2

,timestamp_x,latitude_x,longitude_x,timestamp_y,latitude_y,longitude_y,count_overlaps_in_int,dist,date,time_together
0,2019-09-09 06:39:14.097,52.2525362,0.7138794,2019-09-09 06:46:51.199,52.2525704,0.7138478,2,4.374792876,2019-09-09,03:41:23.498000
1,2019-09-09 06:39:14.097,52.2525362,0.7138794,2019-09-09 06:48:59.484,52.2526024,0.7138553,2,7.547835973,2019-09-09,03:41:23.498000
2,2019-09-09 06:40:55.850,52.2525082,0.7138945,2019-09-09 06:46:51.199,52.2525704,0.7138478,2,7.620557842,2019-09-09,03:41:23.498000
3,2019-09-09 06:40:55.850,52.2525082,0.7138945,2019-09-09 06:48:59.484,52.2526024,0.7138553,2,10.818282131,2019-09-09,03:41:23.498000
4,2019-09-09 06:45:08.096,52.2525082,0.7138945,2019-09-09 06:46:51.199,52.2525704,0.7138478,2,7.620557842,2019-09-09,03:41:23.498000
...,...,...,...,...,...,...,...,...,...,...
129,2019-09-14 23:06:03.000,51.4897918,-0.1158281,2019-09-14 22:56:46.626,51.4897181,-0.1156491,1,14.893132608,2019-09-14,04:40:11.522000
130,2019-09-15 08:43:11.727,51.4897612,-0.1157898,2019-09-15 08:51:56.791,51.4898393,-0.1155317,1,19.921482811,2019-09-15,00:18:32.923000
131,2019-09-15 11:04:09.436,51.5309645,-0.1257586,2019-09-15 11:03:42.148,51.5309472,-0.1255995,4,11.207023835,2019-09-15,00:18:32.923000
132,2019-09-15 11:08:13.895,51.5308573,-0.1258008,2019-09-15 11:03:42.148,51.5309472,-0.1255995,4,17.180610921,2019-09-15,00:18:32.923000


In [49]:
output2.rename(columns = {'timestamp_x':'timestamp_1',
                       'latitude_x':'latitude_1',
                         'longitude_x': 'longitude_1',
                         'timestamp_y':'timestamp_2',
                       'latitude_y':'latitude_2',
                         'longitude_y': 'longitude_2',
                         'count_overlaps_in_int': 'count_overlapping_hits_by_max_time',
                         'dist': 'distance_in_m',
                         'date': 'date',
                         'time_together': 'time_together'}, inplace = True)
output2

,timestamp_1,latitude_1,longitude_1,timestamp_2,latitude_2,longitude_2,count_overlapping_hits_by_max_time,distance_in_m,date,time_together
0,2019-09-09 06:39:14.097,52.2525362,0.7138794,2019-09-09 06:46:51.199,52.2525704,0.7138478,2,4.374792876,2019-09-09,03:41:23.498000
1,2019-09-09 06:39:14.097,52.2525362,0.7138794,2019-09-09 06:48:59.484,52.2526024,0.7138553,2,7.547835973,2019-09-09,03:41:23.498000
2,2019-09-09 06:40:55.850,52.2525082,0.7138945,2019-09-09 06:46:51.199,52.2525704,0.7138478,2,7.620557842,2019-09-09,03:41:23.498000
3,2019-09-09 06:40:55.850,52.2525082,0.7138945,2019-09-09 06:48:59.484,52.2526024,0.7138553,2,10.818282131,2019-09-09,03:41:23.498000
4,2019-09-09 06:45:08.096,52.2525082,0.7138945,2019-09-09 06:46:51.199,52.2525704,0.7138478,2,7.620557842,2019-09-09,03:41:23.498000
...,...,...,...,...,...,...,...,...,...,...
129,2019-09-14 23:06:03.000,51.4897918,-0.1158281,2019-09-14 22:56:46.626,51.4897181,-0.1156491,1,14.893132608,2019-09-14,04:40:11.522000
130,2019-09-15 08:43:11.727,51.4897612,-0.1157898,2019-09-15 08:51:56.791,51.4898393,-0.1155317,1,19.921482811,2019-09-15,00:18:32.923000
131,2019-09-15 11:04:09.436,51.5309645,-0.1257586,2019-09-15 11:03:42.148,51.5309472,-0.1255995,4,11.207023835,2019-09-15,00:18:32.923000
132,2019-09-15 11:08:13.895,51.5308573,-0.1258008,2019-09-15 11:03:42.148,51.5309472,-0.1255995,4,17.180610921,2019-09-15,00:18:32.923000
